In [1]:
import torchvision
import torchvision.transforms.v2 as T
import torch
import torch.nn as nn
import torchmetrics

In [2]:
torch.__version__

'2.9.1+cu128'

In [3]:
toTensor = T.Compose([T.ToImage(), T.ToDtype(torch.float32, scale=True)])

train_and_valid_data = torchvision.datasets.FashionMNIST(
    root="dataset", train=True, transform=toTensor,download=True
)

test_data = torchvision.datasets.FashionMNIST(
    root="dataset", train=False, transform=toTensor, download=True
)

torch.manual_seed(42)
train_data, valid_data = torch.utils.data.random_split(
    train_and_valid_data, [55000, 5000]
)


In [4]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [5]:
X_sample, y_sample = train_data[0]
X_sample.shape, X_sample.dtype

(torch.Size([1, 28, 28]), torch.float32)

In [6]:
train_and_valid_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

#### Building the Classifier

In [7]:
class ImageClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_classes):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_inputs, n_hidden1),
            nn.ReLU(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2, n_classes)
        )

    def forward(self, X):
        return self.mlp(X)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
torch.manual_seed(42)
model = ImageClassifier(n_inputs=1*28*28, n_hidden1=300, n_hidden2=100,
                        n_classes=10).to(device)

xentropy = nn.CrossEntropyLoss()

In [10]:
import torchmetrics

def evaluate_tm(model, data_Loader, metric):
    model.eval()
    metric.reset()
    with torch.inference_mode():
        for X_batch, y_batch in data_Loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            metric.update(y_pred, y_batch)
    return metric.compute()


In [11]:
def train2(model, optimizer, criterion, metric, train_loader, valid_loader, n_epochs):
    history = {"train_losses":[],
               "train_metrics":[],
               "valid_metrics": []}
    for epoch in range(n_epochs):
        total_loss = 0.
        metric.reset()
        for X_batch, y_batch in train_loader:
            model.train()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            metric(y_pred, y_batch)
        mean_loss = total_loss / len(train_loader)
        history["train_losses"].append(mean_loss)
        history["train_metrics"].append(metric.compute().item())
        history["valid_metrics"].append(evaluate_tm(model, valid_loader, metric).item())

        print(f"Epoch: {epoch+1}/{n_epochs},",
              f"Train Loss: {history["train_losses"][-1]:.4f}",
              f"Train metric: {history["train_metrics"][-1]:.4f}",
              f"Valid Metrics: {history["valid_metrics"][-1]:.4f}")
    return history


Un comment the line to train the model and see metrics

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
n_epochs = 20
# _ = train2(model, optimizer, xentropy, accuracy, train_loader, valid_loader, n_epochs)

In [13]:
model.eval()
X_new, y_new = next(iter(valid_loader))
X_new = X_new[:3].to(device)
with torch.inference_mode():
    y_pred_logits = model(X_new)
y_pred = y_pred_logits.argmax(dim = 1)
y_pred

tensor([5, 5, 5], device='cuda:0')

In [14]:
[train_and_valid_data.classes[idx] for idx in y_pred]

['Sandal', 'Sandal', 'Sandal']

In [15]:
y_new[:3]

tensor([7, 4, 2])

In [16]:
import torch.nn.functional as F
y_proba = F.softmax(y_pred_logits, dim = 1)
if device == "mps":
    y_proba = y_proba.cpu()
y_proba.round(decimals=3)

tensor([[0.1040, 0.1010, 0.0960, 0.0950, 0.1010, 0.1060, 0.1010, 0.0930, 0.0990,
         0.1030],
        [0.1020, 0.1000, 0.0990, 0.0970, 0.0930, 0.1150, 0.1000, 0.0960, 0.1000,
         0.0970],
        [0.1010, 0.1040, 0.0990, 0.0920, 0.0940, 0.1160, 0.0970, 0.0970, 0.1010,
         0.0990]], device='cuda:0')

In [17]:
y_top4_logits, y_top4_indices = torch.topk(y_pred_logits, k = 4, dim = 1)
y_top4_proba = F.softmax(y_top4_logits, dim = 1)
y_top4_proba.round(decimals = 3)

tensor([[0.2550, 0.2510, 0.2490, 0.2440],
        [0.2750, 0.2450, 0.2400, 0.2400],
        [0.2740, 0.2470, 0.2400, 0.2390]], device='cuda:0')

In [18]:
y_top4_indices

tensor([[5, 0, 9, 1],
        [5, 0, 1, 6],
        [5, 1, 0, 8]], device='cuda:0')

### Fine-tuning Hyperparameters with optuna

In [ ]:
import optuna

def objective(trial, train_loader, valid_loader):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log = True)
    n_hidden = trial.suggest_int("n_hidden", 20, 300)
    model = ImageClassifier(n_inputs=1*28*28,n_hidden1=n_hidden,
                            n_hidden2=n_hidden,n_classes=10).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    xentropy = nn.CrossEntropyLoss()
    accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes=10)
    accuracy = accuracy.to(device)
    best_validation_accuracy = 0.
    
    for epoch in range(n_epochs):
        history = train2(model, optimizer, xentropy, accuracy, train_loader,
                     valid_loader, n_epochs=1)
        validation_accuracy = max(history["valid_metrics"])
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
        trial.report(validation_accuracy, epoch)

        if trial.should_prune():
            raise optuna.TrialPruned
    
    return best_validation_accuracy

In [27]:
torch.manual_seed(42)
sampler = optuna.samplers.TPESampler(seed=42)
objective_with_data = lambda trial: objective(
    trial, train_loader, valid_loader
)
pruner = optuna.pruners.MedianPruner(n_min_trials=5, n_warmup_steps=0,
                                     interval_steps=1)
study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
study.optimize(objective_with_data, n_trials=5)

[I 2026-01-07 12:34:33,786] A new study created in memory with name: no-name-5e26f268-328e-4550-9168-17b098826ff2


Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1471 Valid Metrics: 0.1860
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.2794 Valid Metrics: 0.3500
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.4109 Valid Metrics: 0.4554
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.5137 Valid Metrics: 0.5560
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.5826 Valid Metrics: 0.6026
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6184 Valid Metrics: 0.6228
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6288 Valid Metrics: 0.6326
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6360 Valid Metrics: 0.6372
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6467 Valid Metrics: 0.6424
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6537 Valid Metrics: 0.6436
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6611 Valid Metrics: 0.6530
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6689 Valid Metrics: 0.6620
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6761 Valid Metrics: 0.6700
Epoch: 1/1, Train Loss: 0.0000 Train m

[I 2026-01-07 12:36:44,003] Trial 0 finished with value: 0.7089999914169312 and parameters: {'learning_rate': 0.00031489116479568613, 'n_hidden': 287}. Best is trial 0 with value: 0.7089999914169312.


Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7196 Valid Metrics: 0.7082
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6157 Valid Metrics: 0.7332
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7863 Valid Metrics: 0.8082
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8179 Valid Metrics: 0.8134
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8311 Valid Metrics: 0.8234
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8403 Valid Metrics: 0.8020
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8464 Valid Metrics: 0.8446
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8510 Valid Metrics: 0.8280
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8563 Valid Metrics: 0.8402
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8603 Valid Metrics: 0.8528
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8642 Valid Metrics: 0.8582
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8662 Valid Metrics: 0.8534
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8696 Valid Metrics: 0.8564
Epoch: 1/1, Train Loss: 0.0000 Train m

[I 2026-01-07 12:38:59,801] Trial 1 finished with value: 0.8677999973297119 and parameters: {'learning_rate': 0.008471801418819975, 'n_hidden': 188}. Best is trial 1 with value: 0.8677999973297119.


...

Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.2610 Valid Metrics: 0.2538
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.0997 Valid Metrics: 0.1028
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1009 Valid Metrics: 0.1050
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1051 Valid Metrics: 0.1124
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1160 Valid Metrics: 0.1270
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1347 Valid Metrics: 0.1496
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1548 Valid Metrics: 0.1652
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1699 Valid Metrics: 0.1800
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1800 Valid Metrics: 0.1872
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1855 Valid Metrics: 0.1916
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1893 Valid Metrics: 0.1960
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.1950 Valid Metrics: 0.2008
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.2000 Valid Metrics: 0.2026
Epoch: 1/1, Train Loss: 0.0000 Train m

[I 2026-01-07 12:43:01,484] Trial 3 finished with value: 0.23960000276565552 and parameters: {'learning_rate': 1.7073967431528103e-05, 'n_hidden': 263}. Best is trial 1 with value: 0.8677999973297119.


Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.2429 Valid Metrics: 0.2396
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.4923 Valid Metrics: 0.6290
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.6582 Valid Metrics: 0.6772
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7115 Valid Metrics: 0.7248
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7551 Valid Metrics: 0.7594
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7833 Valid Metrics: 0.7806
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.7999 Valid Metrics: 0.7862
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8114 Valid Metrics: 0.8064
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8184 Valid Metrics: 0.8130
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8237 Valid Metrics: 0.8200
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8291 Valid Metrics: 0.8206
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8338 Valid Metrics: 0.8150
Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8375 Valid Metrics: 0.8304
Epoch: 1/1, Train Loss: 0.0000 Train m

[I 2026-01-07 12:45:03,202] Trial 4 finished with value: 0.8428000211715698 and parameters: {'learning_rate': 0.002537815508265664, 'n_hidden': 218}. Best is trial 1 with value: 0.8677999973297119.


Epoch: 1/1, Train Loss: 0.0000 Train metric: 0.8556 Valid Metrics: 0.8428


In [28]:
study.best_params

{'learning_rate': 0.008471801418819975, 'n_hidden': 188}

In [29]:
study.best_value

0.8677999973297119

### Saving and loading PyTroch models

In [31]:
torch.save(model.state_dict(), "my_fashion_mnist_weights.pt")
new_model = ImageClassifier(n_inputs=1*28*28, n_hidden1 = 300, n_hidden2 = 100,
                            n_classes = 10)
loaded_weights = torch.load("my_fashion_mnist_weights.pt", weights_only=True)
new_model.load_state_dict(loaded_weights)


<All keys matched successfully>

#### Compiling and Optimizing a PyTroch model

In [32]:
torchscript_model = torch.jit.trace(model, X_new)

### Exercises

13

In [36]:
x = torch.tensor([1.2], requires_grad=True)
y = torch.tensor([3.4], requires_grad=True)

def f(x,y):
    return torch.sin((x**2) * y)

result = f(x,y)
result.backward()

x.grad.item(), y.grad.item()

(1.489864706993103, 0.26291730999946594)

14

In [39]:
class Dense2(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, X):
        z = X @ self.weight.T + self.bias
        return F.relu(z)

In [40]:
torch.manual_seed(42)
dense = Dense2(3,5)
X = torch.randn(2,3)
y_pred = dense(X)
y_pred.shape


torch.Size([2, 5])

In [41]:
y_pred_check = F.relu(X @ dense.weight.T + dense.bias)
torch.allclose(y_pred, y_pred_check)

True

Kaiming Initialization

In [44]:
class Dense3(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        nn.init.kaiming_uniform_(self.weight, nonlinearity="relu")
        self.bias = nn.Parameter(torch.zeros(out_features))
    
    def forward(self, X):
        z = X @ self.weight.T + self.bias
        return F.relu(z)

In [45]:
torch.manual_seed(42)
dense3 = Dense3(3,5)
X = torch.randn(2,3)
y_pred3 = dense3(X)
y_pred3.shape

torch.Size([2, 5])

In [47]:
y_pred_check = F.relu(X @ dense3.weight.T + dense3.bias)
torch.allclose(y_pred3, y_pred_check)

True

15 - CoverType Dataset

In [48]:
from sklearn.datasets import fetch_covtype
covertype = fetch_covtype(data_home="../datasets/", download_if_missing=True)

In [79]:
X, y = covertype.data, covertype.target

In [62]:
class_names = ["Spruce/Fir", "Lodgepole Pine", "Ponderosa Pine", "Cottonwood/Willow","Aspen", "Douglas-fir","Krummholz"]

In [82]:
y[0]

np.int32(4)

In [64]:
class_names[y[0]]

'Douglas-fir'

In [ ]:
# X = torch.from_numpy(X)

In [112]:
# The target values start from 1 
y = y - 1

In [ ]:
# y = y + 1

In [99]:
import pandas as pd

cover_df = pd.DataFrame(X, columns=covertype.feature_names)

In [100]:
cover_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_0,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Soil_Type_0,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We first convert meters columns to kms

In [104]:
cover_df["Elevation"] = cover_df["Elevation"] / 1000
cover_df["Horizontal_Distance_To_Hydrology"] = cover_df["Horizontal_Distance_To_Hydrology"] / 1000
cover_df["Vertical_Distance_To_Hydrology"] = cover_df["Vertical_Distance_To_Hydrology"] / 1000
cover_df["Horizontal_Distance_To_Roadways"] = cover_df["Horizontal_Distance_To_Roadways"] / 1000
cover_df["Horizontal_Distance_To_Fire_Points"] = cover_df["Horizontal_Distance_To_Fire_Points"] / 1000

Then we scale the hillshade columns as these are 0 - 255 indexed.

In [105]:
cover_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_0,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Soil_Type_0,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2.596,51.0,3.0,0.258,0.000,0.510,221.0,232.0,148.0,6.279,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.590,56.0,2.0,0.212,-0.006,0.390,220.0,235.0,151.0,6.225,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.804,139.0,9.0,0.268,0.065,3.180,234.0,238.0,135.0,6.121,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.785,155.0,18.0,0.242,0.118,3.090,238.0,238.0,122.0,6.211,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.595,45.0,2.0,0.153,-0.001,0.391,220.0,234.0,150.0,6.172,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
cover_df["Hillshade_9am"] = cover_df["Hillshade_9am"] / 255
cover_df["Hillshade_Noon"] = cover_df["Hillshade_Noon"] / 255
cover_df["Hillshade_3pm"] = cover_df["Hillshade_3pm"] / 255

In [107]:
cover_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_0,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Soil_Type_0,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2.596,51.0,3.0,0.258,0.000,0.510,0.866667,0.909804,0.580392,6.279,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.590,56.0,2.0,0.212,-0.006,0.390,0.862745,0.921569,0.592157,6.225,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.804,139.0,9.0,0.268,0.065,3.180,0.917647,0.933333,0.529412,6.121,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.785,155.0,18.0,0.242,0.118,3.090,0.933333,0.933333,0.478431,6.211,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.595,45.0,2.0,0.153,-0.001,0.391,0.862745,0.917647,0.588235,6.172,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
X = torch.from_numpy(cover_df.values)

In [118]:
X.shape, y.shape

(torch.Size([581012, 54]), torch.Size([581012]))

In [117]:
y = torch.from_numpy(y)